In [3]:
## Auto Gradient 이해하기

import torch
from torchvision.models import resnet18

model = resnet18(pretrained=True)
data = torch.rand(1,3,64,64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\dnsan/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [4]:
# 순전파 단계
prediction = model(data)

In [6]:
# 역전파 단계
loss = (prediction - labels).sum()
loss.backward()

In [8]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [9]:
# 경사 하강법
optim.step()

In [10]:
## 그렇다면 autograd이 어떻게 변화도를 수집하는지?
# requires_grad 은 autograd에 모든 연산들을 추적해야 한다고 알려줌

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

$$ Q = 3a^3 - b^2 $$

In [11]:
# a와 b로부터 새로운 텐서 Q를 만듦

Q = 3*a**3 - b**2

$$ \text{각 미분값은 } 9a^2, -2b$$

In [12]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [13]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [10]:
## nn 모듈 이해하기

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 5개, 5*5 정사각 Conv 행렬
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 입력 이미지 채널 6개, 출력 채널 16개, 5*5 정사각 Conv 행렬
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine 연산 : y = Wx + b
        # 5*5*16 -> 120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 제곱수라면 하나만으로 특정 가능
        x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # foward가 정의되면 backward는 autograd으로 자동정의
    
net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # 첫번째 conv1의 .weight

10
torch.Size([6, 1, 5, 5])


In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0811, -0.0747,  0.0690,  0.1073, -0.0737, -0.0602, -0.0211, -0.0191,
         -0.0160,  0.0743]], grad_fn=<AddmmBackward>)


In [13]:
#  모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정
# 무작위 값으로 역전파 수행

## 주의
### torch.nn은 미니배치만 지원
### 만약 하나의 샘플만 있다면 input.unsqueeze(0)을 사용해서 (1, 1, 64, 64)로 만들어줘야함

net.zero_grad()
out.backward(torch.randn(1, 10))

In [14]:
## 손실함수

output = net(input)
target = torch.randn(10) # 임의의 라벨
target = target.view(1, -1) # 출력과 같은 shape로 만듦

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2864, grad_fn=<MseLossBackward>)


In [15]:
##  역전파

### error를 역전파하기 위해서 loss.backward 만 해주면 됨
### 단 기존에 계산된 변화도의 값을 누적 시키고 싶지 않다면
### 기존에 계산된 변화도를 0으로 만드는 작업 필요

net.zero_grad() # 모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0516,  0.0065,  0.0126, -0.0146,  0.0175, -0.0239])


In [17]:
### 가중치 갱신하는 방법

# weight = weight(기존 w에 gradient만큼 빼는 방법) - learning_rate * gradient

learning_rate = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
### SGD와 같은 갱신 규칙 사용

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정
optimizer.zero_grad() # 변화도 버퍼 0으로 초기화
output = net(input) # forward 결과를 받음
loss = criterion(output, target) # loss를 SGD로 계산
loss.backward() # 역전파
optimizer.step() # 업데이트 진행